In [ ]:
import numpy as np
import librosa
from optcutfreq import optcutfreq
import os
import random
import pandas as pd

def calcular_frecuencia_corte_ideal(carpeta, num_audios, frecuencia_muestreo=44100):
    nombre_carpeta = os.path.basename(carpeta)

    archivos = [archivo for archivo in os.listdir(carpeta) if archivo.lower().endswith('.wav')]

    # Verificar si hay suficientes archivos de audio en la carpeta
    if len(archivos) < num_audios:
        print("No hay suficientes archivos de audio en la carpeta.")
        return

    # Seleccionar aleatoriamente 'num_audios' archivos de audio
    archivos_a_procesar = random.sample(archivos, num_audios)

    # Crear un DataFrame vacío para almacenar los resultados
    resultados_df = pd.DataFrame(columns=['Nombre del Audio', 'Frecuencia de Corte Óptima'])

    for archivo in archivos_a_procesar:
        # Construir la ruta completa del archivo de audio
        ruta_audio = os.path.join(carpeta, archivo)
        # Cargar el archivo de audio
        audio, sr = librosa.load(ruta_audio, sr=frecuencia_muestreo)
        # Obtener la frecuencia de corte óptima
        fc_opt = optcutfreq(audio, freq=sr, show=True)
        # Devolver el resultado actual como un generador
        yield {'Nombre del Audio': archivo, 'Frecuencia de Corte Óptima': fc_opt}
        # Liberar memoria
        del audio

    # No necesitas guardar el DataFrame si estás utilizando un generador

# Ejemplo de uso
# Obtén el directorio de trabajo actual
directorio_actual = os.getcwd()
carpeta_audios = os.path.join(directorio_actual, 'RZUB04')

num_audios_a_analizar = 30
frecuencia_muestreo = 192000

# Convertir el generador en un DataFrame
resultados_generator = calcular_frecuencia_corte_ideal(carpeta_audios, num_audios_a_analizar, frecuencia_muestreo)
resultados_df = pd.DataFrame(resultados_generator)

# Guardar el DataFrame en un archivo CSV con el nombre de la carpeta
nombre_carpeta = os.path.basename(carpeta_audios)
nombre_archivo_csv = f'./{nombre_carpeta}_cut_off.csv'
resultados_df.to_csv(nombre_archivo_csv, index=False)